In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import csv
from lib import *
%matplotlib inline

In [4]:
folder = '../data/new_data'
user_item_map=pickle.load(open(folder + '/user_item_map.pkl','rb'))
item_id_lookup = pickle.load(open(folder + '/item_id_lookup.pkl','rb'))

In [6]:
bundle_item_map=pickle.load(open(folder + '/bundle_item_map.pkl','rb'))
user_bundle_map=pickle.load(open(folder +'/user_bundle_map.pkl','rb'))

In [7]:
bundle_discount_map = pickle.load(open(folder +'/bundle_discount_map.pkl','rb'))
# bundle_final_price_map = pickle.load(open(folder +'/bundle_final_price_map','rb'))

In [11]:
itemID_tags_map = pickle.load(open(folder +'/itemID_tags_map.pkl','rb'))

In [12]:
tags_set=set()
for itemID in itemID_tags_map:
    tags = itemID_tags_map[itemID]
    for tag in tags:
        tags_set.add(tag)
tag_tagID_map = dict()
tagID_tag_map = dict()
for i,tag in enumerate(tags_set):
    tag_tagID_map[tag] = i 
    tagID_tag_map[i] = tag    
    
def get_feat(tags):
    feat=np.zeros(len(tag_tagID_map))
    for tag in tags:
        feat[tag_tagID_map[tag]]=1
    return feat
    

tag_array = np.array([get_feat(itemID_tags_map[itemID]) for itemID in itemID_tags_map])
print tag_array.shape

(2515, 307)


In [13]:
all_data=[]

for user,bundles in user_bundle_map.items():
    for bundle in bundles:
        all_data.append((user,bundle))
        
all_item_data=[]
for user,items in user_item_map.items():
    for item in items:
        all_item_data.append((user,item))

In [14]:
import random
random.shuffle(all_data)
random.shuffle(all_item_data)

# Training data for bundle for bpr model
training_data=all_data[:int(0.9*len(all_data))]
test_data=all_data[int(0.9*len(all_data)):]

# Training data for items for bpr_item model
training_data_2=all_item_data[:int(0.9*len(all_item_data))]
test_data_2=all_item_data[int(0.9*len(all_item_data)):]

In [15]:
num_users = len(user_item_map)
num_items = len(item_id_lookup)
print num_users, num_items

29633 2515


In [18]:
folder = '../data/new_data'

In [19]:
pickle.dump(tag_array, open(folder + "/tag_array", "wb" ) )

In [21]:
# Generting training data for items through graph sampling.
sgd_train_users_items, sgd_train_pos_items, sgd_train_neg_items\
        = graph_sampling(len(training_data_2)*30, training_data_2, user_item_map)

0
10000000


In [22]:
folder = '../data/train_test'
pickle.dump( sgd_train_users_items, open(folder + "/sgd_train_users_items", "wb" ) )
pickle.dump( sgd_train_pos_items, open(folder +  "/sgd_train_pos_items", "wb" ) )
pickle.dump( sgd_train_neg_items, open(folder +  "/sgd_train_neg_items", "wb" ) )

In [23]:
test_users_cold, test_pos_items_cold, test_neg_items_cold=\
        get_test_data_items(test_data_2, training_data_2)

0


In [24]:
pickle.dump( test_users_cold, open(folder + "/test_users_cold", "wb" ) )
pickle.dump( test_pos_items_cold, open(folder +  "/test_pos_items_cold", "wb" ) )
pickle.dump( test_neg_items_cold, open(folder +  "/test_neg_items_cold", "wb" ) )

In [25]:
print len(sgd_train_pos_items)

20942220


In [15]:
set2 = set([])
for x, y, z in zip(test_users_cold, test_pos_items_cold, test_neg_items_cold):
    set2.add((x,y, z))
print len(test_pos_items_cold), len(set2), len(sgd_train_pos_items), len(training_data_2)

7993816 7993816 24380100 812670


In [26]:
import os
os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=gpu,lib.cnmem=0.7,floatX=float32'

In [42]:
# theano-bpr
#
# Copyright (c) 2014 British Broadcasting Corporation
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import theano, numpy
import theano.tensor as T
import time
import sys
from collections import defaultdict

class BPR_ItemP(object):

    def __init__(self, rank, n_users, n_items, n_tags, tag_array, lambda_u = 0.0025, lambda_i = 0.0025, lambda_j = 0.00025, lambda_d = 0.0025, lambda_p = 0.00025, lambda_q = 0.00025, lambda_bias = 0.0, learning_rate = 0.05):
        
        self._rank = rank
        self._n_users = n_users
        self._n_items = n_items
        self._n_tags = n_tags
        self._lambda_u = lambda_u
        self._lambda_i = lambda_i
        self._lambda_j = lambda_j
        self._lambda_d = lambda_d
        self._lambda_p = lambda_p
        self._lambda_q = lambda_q
        self._lambda_bias = lambda_bias
        self._learning_rate = learning_rate
        self.TG = theano.shared(tag_array.astype('float32'), name='TG')
        self._configure_theano()
        self._generate_train_model_function()
        self._generate_test_model_function()

    def _configure_theano(self):
        """
          Configures Theano to run in fast mode
          and using 32-bit floats. 
        """
        theano.config.mode = 'FAST_RUN'
        theano.config.floatX = 'float32'

    def _generate_train_model_function(self):
        u = T.lvector('u')
        i = T.lvector('i')
        j = T.lvector('j')

        h = numpy.random.random((self._n_items, self._rank))
        b = numpy.random.random(self._n_items)#1*n_items
        x = numpy.random.random((self._n_users, self._n_tags))
#         tg = numpy.random.random((self._n_items, self._n_tags))
        #### needs change ###
        
        self.W = theano.shared(numpy.random.random((self._n_users, self._rank)).astype('float32'), name='W')#P
        self.H = theano.shared(h.astype('float32'), name='H')#Q
        self.theta = theano.shared(x.astype('float32'), name='theta')
        self.B = theano.shared(b.astype('float32'), name='B')#beta
#         self.TG = theano.shared(tg.astype('float32'), name='TG')

        
#         A = theano.shared(tag_matrix.astype("int32"))
        #nd_list = numpy.asarray(item_data)
        #theano_item_data = theano.shared(nd_list)
        #self.A_i = theano.shared(get_feat(theano_item_data[i]['tags']),name='A_i')
        #self.A_j = theano.shared(get_feat(theano_item_data[j]['tags']),name='A_j')
        
        #x_ui = B_i+P_u*Q_i+X_u*A_i
        self.W[u]
        x_ui = T.dot(self.W[u], self.H[i].T).diagonal() + self.B[i] + T.dot(self.theta[u], self.TG[i].T).diagonal()
        x_uj = T.dot(self.W[u], self.H[j].T).diagonal() + self.B[j] + T.dot(self.theta[u], self.TG[j].T).diagonal()
        x_uij = T.nnet.sigmoid(x_ui-x_uj)
        
        obj = T.sum(T.log(x_uij) - self._lambda_u * (self.W[u] ** 2).sum(axis=1) - 
                    self._lambda_i * (self.H[i] ** 2).sum(axis=1) - self._lambda_j * 
                    (self.H[j] ** 2).sum(axis=1) - self._lambda_bias * 
                    (self.B[i] ** 2 + self.B[j] ** 2) - self._lambda_d * (self.theta[u] ** 2).sum(axis=1))
        
        T.printing.Print('still training')
    
        cost = - obj

        g_cost_W = T.grad(cost=cost, wrt=self.W)
        g_cost_H = T.grad(cost=cost, wrt=self.H)
        g_cost_B = T.grad(cost=cost, wrt=self.B)
        g_cost_theta = T.grad(cost=cost, wrt=self.theta)

        updates = [(self.W, self.W - self._learning_rate * g_cost_W), (self.H, self.H - self._learning_rate * g_cost_H), 
                   (self.B, self.B - self._learning_rate * g_cost_B), (self.theta, self.theta - self._learning_rate * g_cost_theta)]
        
                                                  #u, i_p, i_n    
        self.train_model = theano.function(inputs=[u, i, j], outputs=cost, updates=updates)

    
    def train(self, s_users=None, s_pos_items=None, s_neg_items=None, batch_size=1000):
        """
          Trains the BPR Matrix Factorisation model using Stochastic
          Gradient Descent and minibatches over `train_data`.

          `train_data` is an array of (user_index, item_index) tuples.

          We first create a set of random samples from `train_data` for 
          training, of size `epochs` * size of `train_data`.

          We then iterate through the resulting training samples by
          batches of length `batch_size`, and run one iteration of gradient
          descent for the batch.
        """
        if len(s_pos_items) < batch_size:
            sys.stderr.write("WARNING: Batch size is greater than number of training samples, switching to a batch size of %s\n" % str(len(train_data)))
            batch_size = len(s_pos_items)
            
        sgd_users, sgd_pos_items, sgd_neg_items = s_users, s_pos_items, s_neg_items
        n_sgd_samples = len(s_users)
        
        z = 0
        t2 = t1 = t0 = time.time()
        while (z+1)*batch_size < n_sgd_samples:
            self.train_model(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size]
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            sys.stderr.flush()
            t1 = t2
        if n_sgd_samples > 0:
            sys.stderr.write("\nTotal training time %.2f seconds; %e per sample\n" % (t2 - t0, (t2 - t0)/n_sgd_samples))
            sys.stderr.flush()
            
    def _generate_test_model_function(self):
        """
          Computes item predictions for `user_index`.
          Returns an array of prediction values for each item
          in the dataset.
        """
        u = T.lvector('u')
        i = T.lvector('i')
        j = T.lvector('j')
        
        x_ui = T.dot(self.W[u], self.H[i].T).diagonal() + self.B[i] + T.dot(self.theta[u], self.TG[i].T).diagonal()
        x_uj = T.dot(self.W[u], self.H[j].T).diagonal() + self.B[j] + T.dot(self.theta[u], self.TG[j].T).diagonal()
        x_uij = x_ui-x_uj
        
        self.test_model = theano.function(inputs=[u, i, j], outputs=x_uij)
   
    def test_bundle(self, sgd_users, sgd_pos_items, sgd_neg_items, batch_size=1000):
        """
          Computes the Area Under Curve (AUC) on `test_data`.

          `test_data` is an array of (user_index, item_index) tuples.

          During this computation we ignore users and items
          that didn't appear in the training data, to allow
          for non-overlapping training and testing sets.
        """
        
        auc_values = []
        z = 0
        t2 = t1 = t0 = time.time()
        n_sgd_samples = len(sgd_users)
        while (z+1)*batch_size < n_sgd_samples:
            pref_list=self.test_model(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size]
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            t1 = t2
            
            auc = np.sum([1.0 if a>0.0 else 0.0 for a in pref_list])
            auc /= batch_size
            
            auc_values.append(auc)
            sys.stderr.write("\rCurrent AUC mean (%s samples): %0.5f" % (str(z*batch_size), numpy.mean(auc_values)))
            sys.stderr.flush()
        
        sys.stderr.write("\n")
        sys.stderr.flush()
        return numpy.mean(auc_values)

In [43]:
bpr_item = BPR_ItemP(10, num_users, num_items, len(tag_array.T), tag_array)

In [45]:
bpr_item.train(s_users=sgd_train_users_items, s_pos_items=sgd_train_pos_items, s_neg_items=sgd_train_neg_items)

Processed 20942000 ( 100.00% ) in 0.0847 seconds
Total training time 1973.66 seconds; 9.424319e-05 per sample


In [46]:
pickle.dump(bpr_item, open(folder +  "/bpr_item", "wb" ) )

In [47]:
bpr_item.test_bundle(test_users_cold, test_pos_items_cold, test_neg_items_cold)

Current AUC mean (6082000 samples): 0.89513nds


0.89513301545544222

In [21]:
#bpr_item.test_bundle(sgd_train_users_items, sgd_train_pos_items, sgd_train_neg_items)

In [22]:
print 'with price and tags Current AUC mean (20000 samples)'
print 'test:  0.89513301545544224'

with price and tags Current AUC mean (20000 samples)
test:  0.89958560102849794
train: 0.99onds


In [23]:
print 'with price Current AUC mean (20000 samples)'
print 'test:  0.87002nds'
print 'train: 0.88955onds'

with price Current AUC mean (20000 samples)
test:  0.87002nds
train: 0.88955onds


In [24]:
print 'withOUT price Current AUC mean (20000 samples)'
print 'test:  0.86871244911077783'
print 'train: 0.89015816451061747'

withOUT price Current AUC mean (20000 samples)
test:  0.86871244911077783
train: 0.89015816451061747


### original paper BPR

In [25]:
bpr_item0 = BPR_Item(10, num_users, num_items)

In [26]:
bpr_item0.train(s_users=sgd_train_users_items, s_pos_items=sgd_train_pos_items, s_neg_items=sgd_train_neg_items)

Processed 24380000 ( 100.00% ) in 0.0146 seconds
Total training time 419.59 seconds; 1.721048e-05 per sample


In [27]:
bpr_item0.test_bundle(test_users_cold, test_pos_items_cold, test_neg_items_cold)

Current AUC mean (7993000 samples): 0.87024nds


0.87024346303015143

In [28]:
# bpr_item0.test_bundle(sgd_train_users_items, sgd_train_pos_items, sgd_train_neg_items)

## Bundle model begins

In [29]:
# Generting training data for bundles through graph sampling.
sgd_users, sgd_pos_bundles, sgd_neg_bundles = graph_sampling(len(training_data)*30, training_data, user_bundle_map)

0


In [30]:
# max_bundle_size=0
# for bundle,items in bundle_item_map.items():
#     if(len(items)>max_bundle_size):
#         max_bundle_size=len(items)
# print max_bundle_size
max_bundle_size = np.max([len(bundle_item_map[key]) for key in bundle_item_map])

In [31]:
def get_items(bundle_id, max_bundle_size, index):
    item=list(bundle_item_map[bundle_id]);
    for i in range(len(item),max_bundle_size):
        item.append(index)
    return item

sgd_pos_items=[get_items(b_id, max_bundle_size, num_items) for b_id in sgd_pos_bundles]
sgd_neg_items=[get_items(b_id, max_bundle_size, num_items) for b_id in sgd_neg_bundles]

In [32]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics.pairwise import cosine_similarity
def compute_diversity_tags(app_data):
    l=len(app_data)
    app_data=[item_id_lookup[d] for d in app_data]
    count=0.0
    similarity=0.0
    for i in range(l):
        if app_data[i] in item_data_map:
            for j in range(i+1,l):
                if app_data[j] in item_data_map:
                    count+=1
                    similarity+=jaccard_similarity_score(get_feat(item_data_map[app_data[i]]['tags']),
                                                         get_feat(item_data_map[app_data[j]]['tags']))
    if count>0:
        return 1-(similarity/count)
    else:
        return 0.0

def compute_diversity_latent(items, H):
    count=0.0
    similarity=0.0
    for i in range(len(items)):
        for j in range(i + 1, len(items)):
            itemID1, itemID2 = items[i], items[j]
            count+=1
            similarity+=cosine_similarity(H[itemID1].reshape(1,-1),H[itemID2].reshape(1,-1))[0,0]
    if count>0:
        return 1-(similarity/count)
    else:
        return 0.0
    
def compute_diversity(app_data, H):
    if H is not None:
        return compute_diversity_latent(app_data, H)
    else :
        return compute_diversity_tags(app_data)

In [33]:
H_item=bpr_item0.H.eval()

In [34]:
bundle_diversity_map=dict()
for bundle,items in bundle_item_map.items():
    bundle_diversity_map[bundle]=compute_diversity_latent(list(items), H_item)

In [35]:
sgd_pos_len=[len(bundle_item_map[b_id]) for b_id in sgd_pos_bundles]
sgd_neg_len=[len(bundle_item_map[b_id]) for b_id in sgd_neg_bundles]
sgd_pos_diversity=[bundle_diversity_map[b_id] for b_id in sgd_pos_bundles]
sgd_neg_diversity=[bundle_diversity_map[b_id] for b_id in sgd_neg_bundles]
sgd_pos_dicount=[bundle_discount_map[b_id] for b_id in sgd_pos_bundles]
sgd_neg_dicount=[bundle_discount_map[b_id] for b_id in sgd_neg_bundles]
# sgd_pos_price =[bundle_final_price_map[b_id] for b_id in sgd_pos_bundles]
# sgd_neg_price =[bundle_final_price_map[b_id] for b_id in sgd_neg_bundles]

In [36]:
print len(sgd_neg_dicount), len(sgd_pos_diversity), len(sgd_pos_dicount)

2364240 2364240 2364240


In [41]:
def get_test_data_bundles1(test_data, train_data, n_items):
    users=[]
    pos_items=[]
    neg_items=[]
    n1=[]
    n2=[]
    pos_diversity, neg_diversity=[], []
    test_bd_pos_discount, test_bd_neg_discount = [], []
    train_dict, train_users, train_items  = data_to_dict(train_data)
    test_dict, test_users, test_items = data_to_dict(test_data)
    auc_values = []
    z = 0
    for i,user in enumerate(test_dict.keys()):
        if(i%100000==0):
            print i

        if user in train_users: 
            for pos_item in test_dict[user]:
                if pos_item in train_items:
                    for neg_item in train_items:
                        if neg_item not in test_dict[user] and neg_item not in train_dict[user]:
                            pos_diversity.append(bundle_diversity_map[pos_item])
                            neg_diversity.append(bundle_diversity_map[neg_item])
                            users.append(user)
                            pos_items.append(pos_item)
                            neg_items.append(neg_item)
                            n1.append(len(bundle_item_map[pos_item]))
                            n2.append(len(bundle_item_map[neg_item]))
                            test_bd_pos_discount.append(bundle_discount_map[pos_item])
                            test_bd_neg_discount.append(bundle_discount_map[neg_item])
                            

    pos_bunldes = pos_items
    neg_bundles = neg_items
    pos_items=[get_items(b_id, max_bundle_size, n_items) for b_id in pos_bunldes]
    neg_items=[get_items(b_id, max_bundle_size, n_items) for b_id in neg_bundles]
    return users, pos_items, neg_items, n1, n2, pos_diversity, neg_diversity, \
            test_bd_pos_discount, test_bd_neg_discount


In [42]:
test_users, test_pos_items, test_neg_items, test_n1, test_n2, test_pos_diversity, test_neg_diversity,\
test_bd_pos_discount, test_bd_neg_discount=\
                    get_test_data_bundles1(test_data, training_data, num_items)

0


In [43]:
print test_n1[0]

3


In [44]:
print np.shape(bpr_item.H.eval())
H_item = np.concatenate((H_item,np.zeros((1,np.shape(H_item)[1]))),axis=0)
H_item=np.array(H_item).astype('float32')
print np.shape(H_item)

print np.shape(bpr_item.B.eval())
B_item=bpr_item.B.eval()
B_item = np.append(B_item,0)
B_item=np.array(B_item).astype('float32')
print np.shape(B_item)

print np.shape(bpr_item.TG.eval())
TG_item = bpr_item.TG.eval()
TG_item = np.concatenate((TG_item,np.zeros((1,np.shape(TG_item)[1]))),axis=0)
TG_item=np.array(TG_item).astype('float32')
print np.shape(TG_item)

(2819, 10)
(2820, 10)
(2819,)
(2820,)
(2819, 307)
(2820, 307)


In [45]:
class BPR_Cold(object):

    def __init__(self, rank, bundle_size, n_users, n_items, lambda_u = 0.0025, lambda_i = 0.0025, lambda_j = 0.00025, lambda_d = 0.0025, lambda_p = 0.00025, lambda_bias = 0.0, learning_rate = 0.05):
        
        self._rank = rank
        self._bundle_rank = bundle_size + 1
        self._n_users = n_users
        self._n_items = n_items
        self._lambda_u = lambda_u
        self._lambda_i = lambda_i
        self._lambda_j = lambda_j
        self._lambda_d = lambda_d
        self._lambda_p = lambda_p
        self._lambda_bias = lambda_bias
        self._learning_rate = learning_rate
        self._configure_theano()
        self._generate_train_model_item_function()
        self._generate_test_model_function()

    def _configure_theano(self):
        theano.config.mode = 'FAST_RUN'
        theano.config.floatX = 'float32'
    
    def _generate_train_model_item_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')
        n2 = T.lvector('n2')
        di = T.dvector('di')
        dj = T.dvector('dj')
        
        self.W1 = bpr_item.W
        self.H1 = theano.shared(H_item.astype('float32'), name='H')
        self.B1 = theano.shared(B_item.astype('float32'), name='B')
        
        
        self.M1 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M1')
        self.M2 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M2')
        self.K = theano.shared(numpy.random.rand(), name='K')
        self.D = theano.shared(numpy.random.rand(), name='D')
        self.N = theano.shared(numpy.random.random(self._bundle_rank).astype('float32'), name='N')
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] + self.D*di
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] + self.D*dj
        
        x_uij = T.nnet.sigmoid(x_ui-x_uj)
        obj = T.sum(T.log(x_uij) - self._lambda_u * (self.M1 ** 2).sum() - \
                    self._lambda_u * (self.M2 ** 2).sum()  - self._lambda_d * (self.K**2) - self._lambda_d * (self.D**2)\
                    -self._lambda_p * (self.N[n2]**2) - self._lambda_p * (self.N[n1]**2))
        cost = - obj

        g_cost_M1 = T.grad(cost=cost, wrt=self.M1)
        g_cost_M2 = T.grad(cost=cost, wrt=self.M2)
        g_cost_K = T.grad(cost=cost, wrt=self.K)
        g_cost_N = T.grad(cost=cost, wrt=self.N)
        g_cost_D = T.grad(cost=cost, wrt=self.D)
        
        updates = [(self.M1, self.M1 - self._learning_rate * .001* g_cost_M1), (self.M2, self.M2 - self._learning_rate *.001* g_cost_M2), 
                   (self.K, self.K - self._learning_rate * .001*g_cost_K), (self.N, self.N - self._learning_rate *g_cost_N),
                  (self.D, self.D - self._learning_rate * g_cost_D)]

        self.train_model_item = theano.function(inputs=[u, i, j, n1, n2, di, dj], outputs=cost, updates=updates)

    
    def train(self, s_users=None, s_pos_items=None, s_neg_items=None, s_pos_len=None, s_neg_len=None,
             s_pos_diversity=None, s_neg_diversity=None,batch_size=1000):
        
        if len(s_users) < batch_size:
            sys.stderr.write("WARNING: Batch size is greater than number of training samples, switching to a batch size of %s\n" % str(len(train_data)))
            batch_size = len(s_users)
        
        sgd_users, sgd_pos_items, sgd_neg_items = s_users, s_pos_items, s_neg_items
        n_sgd_samples = len(s_users)

        z = 0
        t2 = t1 = t0 = time.time()
        while (z+1)*batch_size < n_sgd_samples:
            
            self.train_model_item(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size],
                s_pos_diversity[z*batch_size: (z+1)*batch_size],
                s_neg_diversity[z*batch_size: (z+1)*batch_size],
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            sys.stderr.flush()
            t1 = t2
        if n_sgd_samples > 0:
            sys.stderr.write("\nTotal training time %.2f seconds; %e per sample\n" % (t2 - t0, (t2 - t0)/n_sgd_samples))
            sys.stderr.flush()
    
    def _generate_test_model_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')
        n2 = T.lvector('n2')
        di = T.dvector('di')
        dj = T.dvector('dj')
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] + self.D*di
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] + self.D*dj 
        
        x_uij = x_ui-x_uj
        self.test_model = theano.function(inputs=[u, i, j, n1, n2, di, dj], outputs=x_uij)
        
    def test_bundle(self, sgd_users, sgd_pos_items, sgd_neg_items, s_pos_len, s_neg_len, s_pos_diversity, s_neg_diversity, batch_size=1000):
        
        auc_values = []
        z = 0
        t2 = t1 = t0 = time.time()
        n_sgd_samples = len(sgd_users)
        while (z+1)*batch_size < n_sgd_samples:
            pref_list=self.test_model(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size],
                s_pos_diversity[z*batch_size: (z+1)*batch_size],
                s_neg_diversity[z*batch_size: (z+1)*batch_size]
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            t1 = t2
            
            auc = np.sum([1.0 if a>0.0 else 0.0 for a in pref_list])
            auc /= batch_size
            
            auc_values.append(auc)
            sys.stderr.write("\rCurrent AUC mean (%s samples): %0.5f" % (str(z*batch_size), numpy.mean(auc_values)))
            sys.stderr.flush()
        
        sys.stderr.write("\n")
        sys.stderr.flush()
        return numpy.mean(auc_values)

In [46]:
bpr_cold = BPR_Cold(10, max_bundle_size, len(user_bundle_map.keys()), num_items)

In [47]:
bpr_cold.train(s_users=sgd_users, s_pos_items=sgd_pos_items, s_neg_items=sgd_neg_items, 
         s_pos_len=sgd_pos_len, s_neg_len=sgd_neg_len, s_pos_diversity=sgd_pos_diversity,
               s_neg_diversity=sgd_neg_diversity)

Processed 2364000 ( 99.99% ) in 0.0561 seconds
Total training time 116.55 seconds; 4.929656e-05 per sample


In [48]:
bpr_cold.test_bundle(test_users, test_pos_items, test_neg_items, test_n1, test_n2, test_pos_diversity, test_neg_diversity)

Current AUC mean (1627000 samples): 0.60576nds


0.6057572218807622

In [49]:
print '0.68585732689210954'
bundle_discount_array = [bundle_discount_map[i] for i in range(len(bundle_discount_map)) if i ]

0.68585732689210954


In [ ]:
print 0.73830120101137786 + 'discount devrivate'
print 0.72651723027375203 + 'no discount derivaite'

In [50]:
class BPR_Buncle1(object):

    def __init__(self, rank, bundle_size, n_users, n_items, n_tags, bunlde_max_id, lambda_u = 0.0025, lambda_i = 0.0025, lambda_j = 0.00025, lambda_d = 0.0025, lambda_p = 0.00025, lambda_A = 0.01, lambda_bias = 0.0, learning_rate = 0.05):
        
        self._rank = rank
        self._bundle_rank = bundle_size + 1
        self._n_users = n_users
        self._n_items = n_items
        self._n_bundle = bunlde_max_id + 1
        self._lambda_u = lambda_u
        self._lambda_i = lambda_i
        self._lambda_j = lambda_j
        self._lambda_d = lambda_d
        self._lambda_p = lambda_p
        self._lambda_bias = lambda_bias
        self._lambda_A = lambda_A
        self._learning_rate = learning_rate
        
        self._n_tags = n_tags
        self._configure_theano()
        self._generate_train_model_item_function()
        self._generate_test_model_function()

    def _configure_theano(self):
        theano.config.mode = 'FAST_RUN'
        theano.config.floatX = 'float32'
    
    def _generate_train_model_item_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')#num of items in pos bundle
        n2 = T.lvector('n2')
        di = T.dvector('di')#cb
        dj = T.dvector('dj')#cb
        bdi = T.dvector('bdi')
        bdj = T.dvector('bdj')
        
        self.W1 = bpr_item.W #Pu
        self.H1 = theano.shared(H_item.astype('float32'), name='H')#Qi
        self.B1 = theano.shared(B_item.astype('float32'), name='B')#Bi
        self.TG1 = theano.shared(A_item.astype('float32'), name='TG')
        self.theta1 = bpr_item.X
        
        self.M1 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M1')#w
        self.M2 = theano.shared(numpy.random.random((self._rank, self._rank)).astype('float64'), name='M2')#u
        self.BDu = theano.shared(numpy.random.random((self._n_users)).astype('float64'), name='bdu')#u
        self.K = theano.shared(numpy.random.rand(), name='K')#k
        self.D = theano.shared(numpy.random.rand(), name='D')#C
        self.N = theano.shared(numpy.random.random(self._bundle_rank).astype('float32'), name='N')#Nb
        self.A1 = theano.shared(numpy.random.random((self._n_tags, self._n_tags)), name='A1')
        self.A2 = theano.shared(numpy.random.random((self._n_tags, self._n_tags)), name='A2')
                                
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] + self.D*di + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[i].sum(axis=1).T/n1)).diagonal() + self.BDu[u] * bdi
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] + self.D*dj + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[j].sum(axis=1).T/n1)).diagonal() + self.BDu[u] * bdj
        
        x_uij = T.nnet.sigmoid(x_ui-x_uj)
        obj = T.sum(T.log(x_uij) - self._lambda_u * (self.M1 ** 2).sum() - \
                    self._lambda_u * (self.M2 ** 2).sum()  - self._lambda_d * (self.K**2) - self._lambda_d * (self.D**2)\
                    -self._lambda_p * (self.N[n2]**2) - self._lambda_p * (self.N[n1]**2)) - self._lambda_A * (self.A1 ** 2).sum() - self._lambda_A * (self.A2 ** 2).sum()
        cost = - obj

        g_cost_M1 = T.grad(cost=cost, wrt=self.M1)
        g_cost_M2 = T.grad(cost=cost, wrt=self.M2)
        g_cost_K = T.grad(cost=cost, wrt=self.K)
        g_cost_N = T.grad(cost=cost, wrt=self.N)
        g_cost_D = T.grad(cost=cost, wrt=self.D)
        g_cost_A1 = T.grad(cost=cost, wrt=self.A1)
        g_cost_A2 = T.grad(cost=cost, wrt=self.A2)
        g_cost_BDu = T.grad(cost=cost, wrt=self.BDu)

        
        updates = [(self.M1, self.M1 - self._learning_rate * .001* g_cost_M1), (self.M2, self.M2 - self._learning_rate *.001* g_cost_M2), 
                   (self.K, self.K - self._learning_rate * .001*g_cost_K), (self.N, self.N - self._learning_rate *g_cost_N),
                  (self.D, self.D - self._learning_rate * g_cost_D), 
                   (self.A1, self.A1-self._learning_rate * .001 * g_cost_A1), (self.A2, self.A2-self._learning_rate * .001 * g_cost_A2),
                  (self.BDu, self.BDu-self._learning_rate * .001 * g_cost_BDu)]

        self.train_model_item = theano.function(inputs=[u, i, j, n1, n2, di, dj, bdi, bdj], outputs=cost, updates=updates)

    
    def train(self, s_users=None, s_pos_items=None, s_neg_items=None, s_pos_len=None, s_neg_len=None,
             s_pos_diversity=None, s_neg_diversity=None, bd_pos_discount = None, bd_neg_discount = None, batch_size=1000):
        
        if len(s_users) < batch_size:
            sys.stderr.write("WARNING: Batch size is greater than number of training samples, switching to a batch size of %s\n" % str(len(train_data)))
            batch_size = len(s_users)
        
        sgd_users, sgd_pos_items, sgd_neg_items = s_users, s_pos_items, s_neg_items
        n_sgd_samples = len(s_users)

        z = 0
        t2 = t1 = t0 = time.time()
        while (z+1)*batch_size < n_sgd_samples:
            
            self.train_model_item(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size],
                s_pos_diversity[z*batch_size: (z+1)*batch_size],
                s_neg_diversity[z*batch_size: (z+1)*batch_size],
                bd_pos_discount[z*batch_size: (z+1)*batch_size],
                bd_neg_discount[z*batch_size: (z+1)*batch_size]
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            sys.stderr.flush()
            t1 = t2
        if n_sgd_samples > 0:
            sys.stderr.write("\nTotal training time %.2f seconds; %e per sample\n" % (t2 - t0, (t2 - t0)/n_sgd_samples))
            sys.stderr.flush()
    
    def _generate_test_model_function(self):
        u = T.lvector('u')
        i = T.lmatrix('i')
        j = T.lmatrix('j')
        n1 = T.lvector('n1')
        n2 = T.lvector('n2')
        di = T.dvector('di')
        dj = T.dvector('dj')
        bdi = T.dvector('bdi')
        bdj = T.dvector('bdj')
        
        x_ui = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[i].sum(axis=1).T/n1)).diagonal() + self.K*(self.B1[i].T/n1).T.sum(axis=1) + self.N[n1] + self.D*di + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[i].sum(axis=1).T/n1)).diagonal() + self.BDu[u] * bdi
        x_uj = T.dot(T.dot(self.W1[u],self.M2), T.dot(self.M1, self.H1[j].sum(axis=1).T/n2)).diagonal() + self.K*(self.B1[j].T/n2).T.sum(axis=1) + self.N[n2] + self.D*dj + T.dot(T.dot(self.theta1[u],self.A1), T.dot(self.A2, self.TG1[j].sum(axis=1).T/n1)).diagonal() + self.BDu[u] * bdj
       
        x_uij = x_ui-x_uj - bdi + bdj
        self.test_model = theano.function(inputs=[u, i, j, n1, n2, di, dj, bdi, bdj], outputs=x_uij)
        
    def test_bundle(self, sgd_users, sgd_pos_items, sgd_neg_items, s_pos_len, s_neg_len,
                    s_pos_diversity, s_neg_diversity, bd_pos_discount, bd_neg_discount, batch_size=1000):
        
        auc_values = []
        z = 0
        t2 = t1 = t0 = time.time()
        n_sgd_samples = len(sgd_users)
        while (z+1)*batch_size < n_sgd_samples:
            pref_list=self.test_model(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size],
                s_pos_len[z*batch_size: (z+1)*batch_size],
                s_neg_len[z*batch_size: (z+1)*batch_size],
                s_pos_diversity[z*batch_size: (z+1)*batch_size],
                s_neg_diversity[z*batch_size: (z+1)*batch_size],
                bd_pos_discount[z*batch_size: (z+1)*batch_size],
                bd_neg_discount[z*batch_size: (z+1)*batch_size]
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            t1 = t2
            
            auc = np.sum([1.0 if a>0.0 else 0.0 for a in pref_list])
            auc /= batch_size
            
            auc_values.append(auc)
            sys.stderr.write("\rCurrent AUC mean (%s samples): %0.5f" % (str(z*batch_size), numpy.mean(auc_values)))
            sys.stderr.flush()
        
        sys.stderr.write("\n")
        sys.stderr.flush()
        return numpy.mean(auc_values)

In [51]:
bunlde_max_id = max([x for x in sgd_pos_bundles])

In [52]:
bpr_bundle = BPR_Buncle1(10, max_bundle_size, len(user_bundle_map.keys()), num_items, \
                        len(tag_array.T), bunlde_max_id)

TypeError: __init__() takes at least 9 arguments (7 given)

In [ ]:
bpr_bundle.train(s_users=sgd_users, s_pos_items=sgd_pos_items, s_neg_items=sgd_neg_items, 
            s_pos_len=sgd_pos_len, s_neg_len=sgd_neg_len, s_pos_diversity=sgd_pos_diversity,
            s_neg_diversity=sgd_neg_diversity, \
            bd_pos_discount = sgd_pos_dicount, bd_neg_discount = sgd_neg_dicount)

In [ ]:
print test_pos_bundles[0]

In [ ]:
bpr_bundle.test_bundle(test_users, test_pos_items, test_neg_items, test_n1, test_n2,\
                       test_pos_diversity, test_neg_diversity, \
                       test_bd_pos_discount, test_bd_neg_discount)